### Load dataset

- `paths:` list of paths of full songs.
- `titles:` list of titles of songs.
- `data:` list of dictionary that contains chorus.

In [ ]:
import pandas as pd
dataset = pd.read_csv('Data/DataWithMusic.csv')
paths = list(dataset['Path'])
titles = list(dataset['Title'])
data = []

### Method to extract chorus

You have to download ffmpeg `<b>conda install -c conda-forge ffmpeg</b>` and use anconda.

*`parametes:`*
- `path:` path of full song.
- `main:` main directory to save chorus.
- `songname:` we take name of song from path.
- `Newpath:` new path to save chorus in it.

    return Newpath to save it in new dataframe

In [1]:
import warnings
warnings.filterwarnings('ignore')
from pychorus import find_and_output_chorus

def extract_chorus(path, main):
    songname = path.split('/', 2)[1].split('.')[0]
    Newpath = main + '/' + songname + '.wav'
    
    chorus = find_and_output_chorus(path, Newpath, 15)
    if chorus == None:
        return None
    
    else:
        return Newpath

### Extract chorus

*Parameters*
- `i:` iterator for looping in every path in paths.

extract method when i from 0 to 376 is popular(Songs/RepeatedPopularChorus) and from 377 to end of data is unpopular(Unpopular/RepeatedUnpopularChorus).

`Note:`

*It took more than 6 hours*

In [ ]:
i = 0
while i < len(paths):
    if i <= 376:
        # Songs/RepeatedPopularChorus directory to collect all popular chours.
        path = extract_chorus(paths[i], 'Songs/RepeatedPopularChorus') 
    else:
        # Unpopular/RepeatedUnpopularChorus directory to collect all unpopular chours.
        path = extract_chorus(paths[i], 'Unpopular/RepeatedUnpopularChorus') 

    data.append({'title': titles[i], 'choruspath': path})
    i += 1

### Convert data to dataframe and save it as csv file

In [ ]:
# At first time we should convert data.
# newData = pd.DataFrame(data)  

# Because I do it in several times, I use this code but I must use code above in the first time.
newData = pd.read_csv('Data/NewData.csv')
for row in data:
    newData = newData.append(row, ignore_index = True)

### Check if doubled choruspath in two songs.

In [ ]:
paths = list(newData['choruspath'])
lost = []
for path in paths:
    if len(newData[newData['choruspath']==path]) > 1:
        lost.append(path)

lost

### Clean data if some songs don't have chours or in lost list

In [ ]:
newData = newData[newData.choruspath.isin(lost) == False]
newData.dropna(inplace=True)

In [ ]:
## I did it because I made code in several days to save my work.
newData.reset_index()
newData.to_csv('Data/newData.csv', index=False)

### Now add newData to dataset and clean it.

In [ ]:
dataset['choruspath'] = newData['choruspath'] # Add path column
dataset['title'] = newData['title'] # Add title column with Title column in newData to check data
newtitles = list(dataset['title'])  # to collect titles which we added
titles = list(dataset['Title'])     # to coolect old Titles

In [ ]:
lostdata = []  # list to Titles and titles with doesn't matches
for i in range(len(titles)):
    if titles[i] != newtitles[i]:
        lostdata.append(titles[i])

# to see if any lost data or not. if(just delete rows) else dataset is good
lostdata

In [ ]:
# delete nan rows and data which titles doesn't match
dataset = dataset[dataset.Title.isin(lostdata) == False]
dataset.dropna(inplace=True)

In [ ]:
# if we deleted rows just reset index after that delete title row
dataset.reset_index()
del dataset['title']

In [ ]:
# Save last data
dataset.to_csv('Data\chorusData.csv', index=False)